In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import time 
from selenium.webdriver import Chrome 

In [5]:
pip install selenium==4.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.0/986.0 kB 668.3 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: selenium
    Found existing installation: selenium 4.9.0
    Uninstalling selenium-4.9.0:
      Successfully uninstalled selenium-4.9.0
Note: you may need to restart the kernel to use updated packages.


In [153]:
# ссылка с которой будем парсить
url = 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=35000000&object_type%5B0%5D=2&offer_type=flat&region=1&room1=1&room2=1&room3=1&room4=1&room7=1&room9=1'


In [155]:
# открываем браузер и переходим по ссылке
driver = webdriver.Chrome()
driver.get(url)


In [156]:
# пробуем достать объявления с одной страницы
flats = driver.find_elements(By.CLASS_NAME, '_93444fe79c--media--9P6wN')
len(flats)

28

In [157]:
# пробуем открыть объявление
page = flats[0].click()

In [158]:
# переходим во вкладку с объявлением 
driver.switch_to.window(driver.window_handles[1])

In [111]:
# пытаемся достать информацию о местонаождении квартиры
address = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--address-line--GRDTb')]
address = address[0].split(',')[1]


In [131]:
# вытаскиваем список метро и время до каждого метро
metro = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--underground--pjGNr')]
metro = [x.split('\n') for x in metro]
metro_res = {}
for i in metro:
    m = int(i[1].split()[0])
    metro_res[i[0]] = m


In [68]:
# в некоторых объявлениях указано шоссе, расположенное неподалеку
highway = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--highway--TV8oi')]
highway 

[]

In [83]:
# вытаскиваем общую информацию о квартире
s = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--text--eplgM')]
s

['Общая площадь\n32,2 м²',
 'Этаж\n4 из 22',
 'Год сдачи\n2025',
 'Дом\nНе сдан',
 'Отделка\nБез отделки']

In [84]:
inf = {}
s=[x.split('\n') for x in s]
for i in s:
    inf[i[0]] = i[1] 
inf

{'Общая площадь': '32,2 м²',
 'Этаж': '4 из 22',
 'Год сдачи': '2025',
 'Дом': 'Не сдан',
 'Отделка': 'Без отделки'}

In [105]:
# целевая переменная
price = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--amount--ON6i1')]
price

['16 042 265 ₽']

In [132]:
# парсим информацию о доме и вытаскиваем оттуда застройщика
builder = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--item--E1gcC')]
builder = builder[1].split('\n')[1]
builder

'Компания Пионер'

In [159]:
# вытаскиваем информацию из шапки объявления (сколько комнат, тип квартиры, площадь, ЖК)
rooms = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--container--pWxZo')]
rooms

['1-комн. квартира, 32,2 м²\nв ЖК «PRIDE (Прайд)»']

In [8]:
driver = webdriver.Chrome('chromedriver_mac64/chromedriver')

/var/folders/k9/wvgj_g6d2k9_5f_qthnr9_f00000gn/T/ipykernel_3292/1372811382.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_mac64/chromedriver')


In [9]:
# сюда будем добавлять инфомацию по каждой квартире, сначала хотела уже в отредактированном виде, но не получилось
# парсила информацию в виде строк, потом будем делить на признаки
inf = []
okrug = []
metro = []
road = []
flat_inf = []
developer = []
y =[]


In [30]:
def parcer_cian(number):
    driver = webdriver.Chrome('chromedriver_mac64/chromedriver')
    url = f'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=35000000&object_type%5B0%5D=2&offer_type=flat&p={number}&region=1&room1=1&room2=1&room3=1&room4=1&room7=1&room9=1'
    driver.get(url)
    time.sleep(5) 
    flats = driver.find_elements(By.CLASS_NAME, '_93444fe79c--media--9P6wN')
    for flat in flats:
        flat.click()
        time.sleep(5) 
        driver.switch_to.window(driver.window_handles[1])
        rooms = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--container--pWxZo')]
        inf.append(rooms)
        address = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--address-line--GRDTb')]
        okrug.append(address)
        metro_res = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--underground--pjGNr')]
        metro.append(metro_res)
        highway = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--highway--TV8oi')]
        road.append(highway) 
        s = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--text--eplgM')]
        flat_inf.append(s)
        builder = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--item--E1gcC')]
        developer.append(builder)
        price = [x.text for x in driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--amount--ON6i1')]
        y.append(price)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    driver.close()

In [10]:
parcer_cian(1)

In [31]:
for i in range(3,50):
    parcer_cian(i)

/var/folders/k9/wvgj_g6d2k9_5f_qthnr9_f00000gn/T/ipykernel_3292/2293598058.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_mac64/chromedriver')


IndexError: list index out of range

In [142]:
flat_inf[:5]

[['Общая площадь\n32,2 м²',
  'Этаж\n5 из 22',
  'Год сдачи\n2025',
  'Дом\nНе сдан',
  'Отделка\nБез отделки'],
 ['Общая площадь\n48,06 м²',
  'Этаж\n2 из 26',
  'Год сдачи\n2023',
  'Дом\nНе сдан',
  'Отделка\nНеизвестно'],
 ['Общая площадь\n37,62 м²',
  'Жилая площадь\n12,8 м²',
  'Площадь кухни\n3,1 м²',
  'Этаж\n2 из 36',
  'Год сдачи\n2025',
  'Дом\nНе сдан',
  'Отделка\nЧистовая'],
 ['Общая площадь\n42,8 м²',
  'Этаж\n2 из 45',
  'Год сдачи\n2026',
  'Дом\nНе сдан',
  'Отделка\nБез отделки'],
 ['Общая площадь\n47,9 м²',
  'Этаж\n2 из 16',
  'Год сдачи\n2023',
  'Дом\nСдан',
  'Отделка\nПредчистовая']]

In [38]:
inf[:10]

[['1-комн. квартира, 32,2 м²\nв ЖК «PRIDE (Прайд)»'],
 ['1-комн. квартира, 48,06 м²\nв ЖК «ÁLIA (АЛИЯ)»'],
 ['1-комн. квартира, 37,62 м²\nв ЖК «SYMPHONY 34 (Симфони 34)»'],
 ['1-комн. квартира, 42,8 м²\nв ЖК «River Park Towers Кутузовский (Ривер Парк Тауэрс Кутузовский)»'],
 ['1-комн. апартаменты, 47,9 м²\nв ЖК «Комплекс апартаментов Slava (Слава)»'],
 ['1-комн. квартира, 44,9 м²\nв ЖК «Остров»'],
 [],
 ['3-комн. квартира, 85,13 м²\nв ЖК «SYMPHONY 34 (Симфони 34)»'],
 ['1-комн. апартаменты, 48,73 м²\nв ЖК «Lunar (Лунар)»'],
 ['2-комн. апартаменты, 77,85 м²\nв ЖК «Глоракс Премиум Белорусская (GloraX Premium Белорусская)»']]

In [40]:
okrug[:10]

[['Москва, СВАО, р-н Марьина роща, Полковая ул., 1С5На карте'],
 ['Москва, СЗАО, р-н Покровское-Стрешнево, Алиа жилой комплекс, к6На карте'],
 ['Москва, САО, р-н Савеловский, Симфония 34 жилой комплексНа карте'],
 ['Москва, ЗАО, р-н Дорогомилово, Кутузовский проезд, 16А/1На карте'],
 ['Москва, САО, р-н Беговой, Северный административный округ, Слава жилой комплекс, к2На карте'],
 ['Москва, СЗАО, р-н Хорошево-Мневники, проезд Проектируемый № 1077На карте'],
 [],
 ['Москва, САО, р-н Савеловский, 2-я Хуторская ул., 34На карте'],
 ['Москва, ЮЗАО, р-н Гагаринский, Лунар жилой комплексНа карте'],
 ['Москва, САО, р-н Беговой, ул. 1-я Ямского Поля, 28С3На карте']]

In [42]:
metro[:10]

[['Савёловская\n13 мин.', 'Марьина роща\n17 мин.', 'Савёловская\n18 мин.'],
 ['Спартак\n7 мин.', 'Тушинская\n13 мин.', 'Трикотажная\n4 мин.'],
 ['Дмитровская\n8 мин.', 'Гражданская\n16 мин.', 'Тимирязевская\n3 мин.'],
 ['Фили\n5 мин.', 'Парк Победы\n12 мин.', 'Кутузовская\n17 мин.'],
 ['Белорусская\n4 мин.', 'Маяковская\n16 мин.', 'Менделеевская\n20 мин.'],
 ['Терехово\n1 мин.', 'Мнёвники\n4 мин.', 'Карамышевская\n5 мин.'],
 [],
 ['Дмитровская\n7 мин.', 'Гражданская\n17 мин.', 'Тимирязевская\n3 мин.'],
 ['Ленинский проспект\n13 мин.',
  'Площадь Гагарина\n15 мин.',
  'Воробьёвы горы\n19 мин.'],
 ['Белорусская\n7 мин.', 'Менделеевская\n18 мин.', 'Савёловская\n18 мин.']]

# Оформление данных в csv файл

### Целевая переменная

Из в списке из списков цен вытащим цены без значка рубли и запишем в новый датафрейм

In [67]:

new_y = []
for i in y:
    if i != []:
        new_y.append(i[0][:-2])
    else:
        new_y.append('')
new_y[:10]

['16 106 665',
 '16 122 800',
 '18 472 662',
 '22 450 632',
 '25 160 002',
 '25 606 470',
 '',
 '30 781 218',
 '31 000 000',
 '34 069 999']

In [68]:
import pandas as pd
target = pd.DataFrame({'Цена': new_y})

In [70]:
target.head()

,Цена
0,16 106 665
1,16 122 800
2,18 472 662
3,22 450 632
4,25 160 002


### Количество комнат, тип квартиры, название ЖК

Вытащим количество комнат и название ЖК

In [71]:
inf[:10]

[['1-комн. квартира, 32,2 м²\nв ЖК «PRIDE (Прайд)»'],
 ['1-комн. квартира, 48,06 м²\nв ЖК «ÁLIA (АЛИЯ)»'],
 ['1-комн. квартира, 37,62 м²\nв ЖК «SYMPHONY 34 (Симфони 34)»'],
 ['1-комн. квартира, 42,8 м²\nв ЖК «River Park Towers Кутузовский (Ривер Парк Тауэрс Кутузовский)»'],
 ['1-комн. апартаменты, 47,9 м²\nв ЖК «Комплекс апартаментов Slava (Слава)»'],
 ['1-комн. квартира, 44,9 м²\nв ЖК «Остров»'],
 [],
 ['3-комн. квартира, 85,13 м²\nв ЖК «SYMPHONY 34 (Симфони 34)»'],
 ['1-комн. апартаменты, 48,73 м²\nв ЖК «Lunar (Лунар)»'],
 ['2-комн. апартаменты, 77,85 м²\nв ЖК «Глоракс Премиум Белорусская (GloraX Premium Белорусская)»']]

In [376]:
rooms = []
name_jk = []
for j in inf:
    if j != []:
        j = j[0].split('\n')
        if len(j)==2:
            rooms.append(j[0])
            name_jk.append(j[1][1:])
        else:
            rooms.append(j[0])
            name_jk.append('')
    else:
        rooms.append('')
        name_jk.append('')

In [377]:
rooms = [x.split(', ')[0] for x in rooms]

In [379]:
name_jk_latest = []
for jk in name_jk:
    if jk != []:
        name_jk_latest.append(' '.join(jk.split()[1:]))
    else:
        name_jk_latest.append('')

In [111]:
rooms[:10]

['1-комн. квартира',
 '1-комн. квартира',
 '1-комн. квартира',
 '1-комн. квартира',
 '1-комн. апартаменты',
 '1-комн. квартира',
 '',
 '3-комн. квартира',
 '1-комн. апартаменты',
 '2-комн. апартаменты']

In [380]:
name_jk_latest[:10]

['«PRIDE (Прайд)»',
 '«ÁLIA (АЛИЯ)»',
 '«SYMPHONY 34 (Симфони 34)»',
 '«River Park Towers Кутузовский (Ривер Парк Тауэрс Кутузовский)»',
 '«Комплекс апартаментов Slava (Слава)»',
 '«Остров»',
 '',
 '«SYMPHONY 34 (Симфони 34)»',
 '«Lunar (Лунар)»',
 '«Глоракс Премиум Белорусская (GloraX Premium Белорусская)»']

C названиями ЖК мы разобрались, теперь перейдем к количеству комнат в квартире и типу квартиры. На момент выполнения данного задания наша команда считает, что тип (квартира, апартаменты , студия) оказывает влияние на конечную цену (мы хотим взять это за гипотезу), поэтому список rooms на количество комнат в квартире и тип квартиры. 

Проверим какие уникальные значения есть в списке rooms, чтобы понять как разделить 

In [119]:
set(rooms)

{'',
 '1-комн. апартаменты',
 '1-комн. квартира',
 '2-комн. апартаменты',
 '2-комн. квартира',
 '3-комн. апартаменты',
 '3-комн. квартира',
 '4-комн. апартаменты',
 '4-комн. квартира',
 'Апартаменты-студия',
 'Студия'}

In [136]:
rooms_type_flat = [x.split() for x in rooms]
rooms_type_flat[:10]

[['1-комн.', 'квартира'],
 ['1-комн.', 'квартира'],
 ['1-комн.', 'квартира'],
 ['1-комн.', 'квартира'],
 ['1-комн.', 'апартаменты'],
 ['1-комн.', 'квартира'],
 [],
 ['3-комн.', 'квартира'],
 ['1-комн.', 'апартаменты'],
 ['2-комн.', 'апартаменты']]

In [138]:
num_rooms = []
type_flat = []

for k in rooms_type_flat:
    if k != []:
        if sum(list(map(str.isdigit, k[0]))) !=0 : # есть ли в строке есть цифра
            num_rooms.append(int(k[0][0]))
            type_flat.append(k[1])
        else:
            num_rooms.append('')
            type_flat.append(k[0])
    else:
        num_rooms.append('')
        type_flat.append('')

In [139]:
num_rooms[:10]

[1, 1, 1, 1, 1, 1, '', 3, 1, 2]

In [141]:
type_flat[:10]

['квартира',
 'квартира',
 'квартира',
 'квартира',
 'апартаменты',
 'квартира',
 '',
 'квартира',
 'апартаменты',
 'апартаменты']

### Застройщик, класс, тип дома, парковка, отделка

In [145]:
max(developer, key = len)

['Сдача комплекса\nСдача в 4 кв. 2025',
 'Застройщик\nКомпания Пионер',
 'Класс\nПремиум',
 'Кол-во корпусов\n1 корпус',
 'Тип дома\nМонолитный',
 'Парковка\nПодземная',
 'Отделка\nБез отделки, чистовая']

Напишем функцию, которая будет преобразовывать неудобные списки вида "характеристика\nзначение" в список из словарей вида характеристика : значение

In [279]:
def list_to_dict(list_to_change):
    new_list = []
    for i in list_to_change:
        d = {}
        i_split = [y.split('\n') for y in i]
        for j in i_split:
            d[j[0]] = j[1]
        new_list.append(d)
    return new_list

In [282]:
total_inf_houses = list_to_dict(developer)


In [283]:
release_year = [] 
build_company = []
class_liv = []
house_type = []
parking = []
decor = []

In [284]:
for house in total_inf_houses:
    if 'Сдача комплекса' in house.keys():
        release_year.append(house['Сдача комплекса'])
    else:
        release_year.append('')
    if 'Застройщик' in house.keys():
        build_company.append(house['Застройщик'])
    else:
        build_company.append('')
    if 'Класс' in house.keys():
        class_liv.append(house['Класс'])
    else:
        class_liv.append('')
    if 'Тип дома' in house.keys():
        house_type.append(house['Тип дома'])
    else:
        house_type.append('')
    if 'Парковка' in house.keys():
        parking.append(house['Парковка'])
    else:
        parking.append('')
    if 'Отделка' in house.keys():
        decor.append(house['Отделка'])
    else:
        decor.append('')

In [285]:
set(build_company)

{'',
 '3SGroup',
 '494 УНР',
 'AFI Development',
 'Absolute Premium',
 'Forma',
 'GRAVION',
 'Glincom',
 'GloraX',
 'Hutton Development',
 'Level Group',
 'MR Group',
 'October Group',
 'Optima Development',
 'Prime Life Development',
 'Regions Development',
 'Seven Suns Development',
 'Sezar Group',
 'Sminex-Интеко',
 'St Michael',
 'TEKTA GROUP',
 'Upside Development',
 'VOS’HOD',
 'Wainbridge',
 'Ziggurat',
 'А101',
 'Абсолют Недвижимость',
 'Аеон Девелопмент',
 'Велесстрой',
 'ГК МИЦ',
 'ГК Новая Жизнь',
 'ГК Основа',
 'ГК ФСК',
 'Группа ЛСР',
 'Группа Родина',
 'Группа Самолет',
 'Группа Эталон',
 'ДОНСТРОЙ',
 'ДСК-1',
 'КОРТРОС',
 'Колди (Coldy)',
 'Компания Пионер',
 'Концерн КРОСТ',
 'Мангазея',
 'Новые Ватутинки',
 'ООО "СЗ "Стадион "Спартак"',
 'ПИК',
 'РГ-Девелопмент',
 'СЗ Звездный',
 'СЗ Логос',
 'СЗ ПРОГРЕСС',
 'СЗ Универсаль',
 'СИТИ21',
 'СК Резиденс',
 'Ташир',
 'Э.К. Девелопмент'}

In [286]:
set(class_liv )

{'', 'Бизнес', 'Комфорт', 'Премиум'}

In [287]:
set(house_type )

{'',
 'Монолитно-кирпичный',
 'Монолитно-кирпичный, монолитный',
 'Монолитно-кирпичный, монолитный, кирпичный',
 'Монолитный',
 'Панельный',
 'Панельный, кирпичный',
 'Панельный, монолитный'}

In [288]:
set(parking )

{'',
 '-',
 'Гостевая',
 'Отдельная многоуровневая',
 'Отдельная многоуровневая, гостевая',
 'Подземная',
 'Подземная, гостевая',
 'Подземная, отдельная многоуровневая',
 'Подземная, отдельная многоуровневая, гостевая'}

In [289]:
set(decor )

{'',
 'Без отделки',
 'Без отделки, предчистовая',
 'Без отделки, предчистовая, черновая',
 'Без отделки, предчистовая, черновая, чистовая',
 'Без отделки, предчистовая, чистовая',
 'Без отделки, черновая',
 'Без отделки, черновая, чистовая',
 'Без отделки, чистовая',
 'Без отделки, чистовая, чистовая с мебелью',
 'Предчистовая',
 'Предчистовая, черновая',
 'Предчистовая, черновая, чистовая',
 'Предчистовая, чистовая',
 'Черновая',
 'Чистовая',
 'Чистовая, чистовая с мебелью'}

В принципе все признаки выглядят +- адекватно, кроме года сдачи, нужно почистить строку и оставить только год

In [178]:
set(release_year)

{'',
 'Сдан',
 'Сдача в 1 кв. 2024',
 'Сдача в 1 кв. 2024, есть сданные',
 'Сдача в 1 кв. 2025',
 'Сдача в 1 кв. 2025, есть сданные',
 'Сдача в 1 кв. 2026',
 'Сдача в 1—2 кв. 2024, есть сданные',
 'Сдача в 1—3 кв. 2024, есть сданные',
 'Сдача в 2 кв. 2023',
 'Сдача в 2 кв. 2023, есть сданные',
 'Сдача в 2 кв. 2024',
 'Сдача в 2 кв. 2024, есть сданные',
 'Сдача в 2 кв. 2025',
 'Сдача в 2 кв. 2025, есть сданные',
 'Сдача в 2 кв. 2026',
 'Сдача в 2023, есть сданные',
 'Сдача в 2023—2024',
 'Сдача в 2023—2024, есть сданные',
 'Сдача в 2023—2025',
 'Сдача в 2023—2025, есть сданные',
 'Сдача в 2023—2026, есть сданные',
 'Сдача в 2023—2027, есть сданные',
 'Сдача в 2024—2025',
 'Сдача в 2024—2025, есть сданные',
 'Сдача в 2024—2026',
 'Сдача в 2024—2026, есть сданные',
 'Сдача в 2025',
 'Сдача в 2025—2026',
 'Сдача в 2026—2027',
 'Сдача в 2—3 кв. 2023, есть сданные',
 'Сдача в 3 кв. 2023',
 'Сдача в 3 кв. 2023, есть сданные',
 'Сдача в 3 кв. 2024',
 'Сдача в 3 кв. 2024, есть сданные',
 'Сдача

In [187]:
release_year_last = [x.split(',')[0] for x in release_year]
release_year_last[:10]

['Сдача в 4 кв. 2025',
 'Сдача в 2023—2025',
 'Сдача в 2 кв. 2025',
 'Сдача в 2026—2027',
 'Сдан',
 '',
 '',
 'Сдача в 2 кв. 2025',
 'Сдача в 4 кв. 2023',
 'Сдача в 3 кв. 2024']

In [186]:
set(release_year_last)

{'',
 'Сдан',
 'Сдача в 1 кв. 2024',
 'Сдача в 1 кв. 2025',
 'Сдача в 1 кв. 2026',
 'Сдача в 1—2 кв. 2024',
 'Сдача в 1—3 кв. 2024',
 'Сдача в 2 кв. 2023',
 'Сдача в 2 кв. 2024',
 'Сдача в 2 кв. 2025',
 'Сдача в 2 кв. 2026',
 'Сдача в 2023',
 'Сдача в 2023—2024',
 'Сдача в 2023—2025',
 'Сдача в 2023—2026',
 'Сдача в 2023—2027',
 'Сдача в 2024—2025',
 'Сдача в 2024—2026',
 'Сдача в 2025',
 'Сдача в 2025—2026',
 'Сдача в 2026—2027',
 'Сдача в 2—3 кв. 2023',
 'Сдача в 3 кв. 2023',
 'Сдача в 3 кв. 2024',
 'Сдача в 3 кв. 2025',
 'Сдача в 3 кв. 2026',
 'Сдача в 4 кв. 2023',
 'Сдача в 4 кв. 2024',
 'Сдача в 4 кв. 2025',
 'Сдача в 4 кв. 2026'}

По списку уникальных значений видно, что у нас есть варианты, где прописан квартал и единственный год, диапазон лет и уже сдан

In [197]:
year_of_release = []
for year in release_year_last:
    if year != '':
        year_of_release.append(year.split()[-1])
    else:
        year_of_release.append('')

In [199]:
 set(year_of_release)

{'',
 '2023',
 '2023—2024',
 '2023—2025',
 '2023—2026',
 '2023—2027',
 '2024',
 '2024—2025',
 '2024—2026',
 '2025',
 '2025—2026',
 '2026',
 '2026—2027',
 'Сдан'}

### Информация о квартире (площадь, количество комнат, отделка и тд)

Находим самый длинный список, чтобы понять сколько всего характеристик возможно. Далее придадим каждому списку более читабельную форму, то есть в виде словаря Характеристика: значения. 

In [202]:
max(flat_inf, key = len)

['Общая площадь\n37,62 м²',
 'Жилая площадь\n12,8 м²',
 'Площадь кухни\n3,1 м²',
 'Этаж\n2 из 36',
 'Год сдачи\n2025',
 'Дом\nНе сдан',
 'Отделка\nЧистовая']

In [203]:
total_inf_flats = []

for i in flat_inf:
    flat = {}
    flat_split = [y.split('\n') for y in i]
    for j in flat_split:
        flat[j[0]] = j[1]
    total_inf_flats.append(flat)

In [207]:
total_inf_flats[2]

{'Общая площадь': '37,62 м²',
 'Жилая площадь': '12,8 м²',
 'Площадь кухни': '3,1 м²',
 'Этаж': '2 из 36',
 'Год сдачи': '2025',
 'Дом': 'Не сдан',
 'Отделка': 'Чистовая'}

Теперь для каждой характеристики сделаем словарь и будем добавлять туда значение каждой квартиры по ключу, а если такого ключа нет, то пустое значение

In [210]:
square_all = []
square_liv = []
square_kitch = []
floors_flat_all = []
god_sdachi = []
decor_flat = []

In [211]:
for flat in total_inf_flats:
    if 'Общая площадь' in flat.keys():
        square_all.append(flat['Общая площадь'])
    else:
        square_all.append('')
    if 'Жилая площадь' in flat.keys():
        square_liv .append(flat['Жилая площадь'])
    else:
        square_liv .append('')
    if 'Площадь кухни' in flat.keys():
        square_kitch.append(flat['Площадь кухни'])
    else:
        square_kitch.append('')
    if 'Этаж' in flat.keys():
        floors_flat_all.append(flat['Этаж'])
    else:
        floors_flat_all.append('')
    if 'Год сдачи' in flat.keys():
        god_sdachi.append(flat['Год сдачи'])
    else:
        god_sdachi.append('')
    if 'Отделка' in flat.keys():
        decor_flat.append(flat['Отделка'])
    else:
        decor_flat.append('')

In [224]:
set(square_all)


{'',
 '103,18 м²',
 '105,4 м²',
 '106,52 м²',
 '108,08 м²',
 '108,2 м²',
 '110 м²',
 '111,86 м²',
 '15,9 м²',
 '17,7 м²',
 '17,83 м²',
 '17,9 м²',
 '18,2 м²',
 '18,6 м²',
 '18,77 м²',
 '19,3 м²',
 '19,6 м²',
 '19,7 м²',
 '19,8 м²',
 '19,85 м²',
 '19,9 м²',
 '19,97 м²',
 '20 м²',
 '20,01 м²',
 '20,08 м²',
 '20,1 м²',
 '20,11 м²',
 '20,12 м²',
 '20,17 м²',
 '20,2 м²',
 '20,3 м²',
 '20,4 м²',
 '20,5 м²',
 '20,6 м²',
 '20,7 м²',
 '20,8 м²',
 '20,9 м²',
 '21 м²',
 '21,02 м²',
 '21,1 м²',
 '21,2 м²',
 '21,3 м²',
 '21,34 м²',
 '21,5 м²',
 '21,6 м²',
 '21,7 м²',
 '21,8 м²',
 '21,9 м²',
 '22 м²',
 '22,11 м²',
 '22,3 м²',
 '22,31 м²',
 '22,4 м²',
 '22,5 м²',
 '22,57 м²',
 '22,7 м²',
 '22,8 м²',
 '22,9 м²',
 '23 м²',
 '23,1 м²',
 '23,2 м²',
 '23,3 м²',
 '23,4 м²',
 '23,46 м²',
 '23,5 м²',
 '23,58 м²',
 '23,6 м²',
 '23,7 м²',
 '23,77 м²',
 '23,78 м²',
 '23,8 м²',
 '23,89 м²',
 '23,9 м²',
 '23,95 м²',
 '24 м²',
 '24,1 м²',
 '24,2 м²',
 '24,35 м²',
 '24,4 м²',
 '24,5 м²',
 '24,52 м²',
 '24,6 м²',
 '

In [226]:
set(square_liv)

{'',
 '0,3 м²',
 '1,2 м²',
 '10 м²',
 '10,1 м²',
 '10,2 м²',
 '10,3 м²',
 '10,4 м²',
 '10,5 м²',
 '10,6 м²',
 '10,7 м²',
 '10,8 м²',
 '10,9 м²',
 '11 м²',
 '11,1 м²',
 '11,2 м²',
 '11,3 м²',
 '11,4 м²',
 '11,5 м²',
 '11,6 м²',
 '11,7 м²',
 '11,8 м²',
 '11,9 м²',
 '12 м²',
 '12,1 м²',
 '12,2 м²',
 '12,3 м²',
 '12,4 м²',
 '12,5 м²',
 '12,6 м²',
 '12,7 м²',
 '12,8 м²',
 '12,9 м²',
 '13,1 м²',
 '13,2 м²',
 '13,3 м²',
 '13,4 м²',
 '13,5 м²',
 '13,6 м²',
 '13,7 м²',
 '13,8 м²',
 '13,9 м²',
 '14 м²',
 '14,1 м²',
 '14,2 м²',
 '14,3 м²',
 '14,4 м²',
 '14,5 м²',
 '14,6 м²',
 '14,7 м²',
 '14,8 м²',
 '14,9 м²',
 '15 м²',
 '15,2 м²',
 '15,3 м²',
 '15,4 м²',
 '15,5 м²',
 '15,6 м²',
 '15,7 м²',
 '15,8 м²',
 '15,9 м²',
 '16 м²',
 '16,1 м²',
 '16,2 м²',
 '16,3 м²',
 '16,4 м²',
 '16,5 м²',
 '16,6 м²',
 '16,7 м²',
 '16,8 м²',
 '16,9 м²',
 '17 м²',
 '17,2 м²',
 '17,3 м²',
 '17,4 м²',
 '17,5 м²',
 '17,7 м²',
 '18 м²',
 '18,1 м²',
 '18,3 м²',
 '18,4 м²',
 '18,5 м²',
 '18,6 м²',
 '18,8 м²',
 '19 м²',
 '19,1 

In [227]:
set(floors_flat_all)

{'',
 '1 из 21',
 '1 из 6',
 '1 из 8',
 '1 из 9',
 '10 из 12',
 '10 из 14',
 '10 из 15',
 '10 из 16',
 '10 из 17',
 '10 из 18',
 '10 из 19',
 '10 из 20',
 '10 из 21',
 '10 из 25',
 '10 из 28',
 '10 из 30',
 '10 из 32',
 '10 из 33',
 '10 из 41',
 '10 из 55',
 '11 из 11',
 '11 из 13',
 '11 из 14',
 '11 из 15',
 '11 из 20',
 '11 из 21',
 '11 из 22',
 '11 из 23',
 '11 из 24',
 '11 из 25',
 '11 из 27',
 '11 из 28',
 '11 из 29',
 '11 из 30',
 '11 из 31',
 '11 из 32',
 '11 из 33',
 '11 из 52',
 '11 из 53',
 '11 из 54',
 '12 из 12',
 '12 из 13',
 '12 из 14',
 '12 из 15',
 '12 из 16',
 '12 из 17',
 '12 из 18',
 '12 из 19',
 '12 из 20',
 '12 из 21',
 '12 из 22',
 '12 из 23',
 '12 из 24',
 '12 из 25',
 '12 из 28',
 '12 из 29',
 '12 из 31',
 '12 из 33',
 '12 из 34',
 '12 из 35',
 '12 из 38',
 '12 из 49',
 '13 из 14',
 '13 из 17',
 '13 из 19',
 '13 из 20',
 '13 из 21',
 '13 из 22',
 '13 из 23',
 '13 из 24',
 '13 из 25',
 '13 из 26',
 '13 из 28',
 '13 из 29',
 '13 из 30',
 '13 из 33',
 '13 из 39',
 

In [220]:
set(god_sdachi)

{'', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026'}

In [225]:
set(decor_flat)

{'',
 'Без отделки',
 'Неизвестно',
 'Предчистовая',
 'С отделкой',
 'Черновая',
 'Чистовая',
 'Чистовая с мебелью'}

In [228]:
set(square_kitch)

{'',
 '1,2 м²',
 '1,9 м²',
 '10 м²',
 '10,1 м²',
 '10,2 м²',
 '10,3 м²',
 '10,4 м²',
 '10,5 м²',
 '10,6 м²',
 '10,7 м²',
 '10,8 м²',
 '10,9 м²',
 '11 м²',
 '11,1 м²',
 '11,2 м²',
 '11,3 м²',
 '11,4 м²',
 '11,7 м²',
 '11,8 м²',
 '11,9 м²',
 '12 м²',
 '12,1 м²',
 '12,2 м²',
 '12,3 м²',
 '12,4 м²',
 '12,5 м²',
 '12,6 м²',
 '12,7 м²',
 '12,8 м²',
 '12,9 м²',
 '13 м²',
 '13,1 м²',
 '13,2 м²',
 '13,3 м²',
 '13,4 м²',
 '13,5 м²',
 '13,6 м²',
 '13,7 м²',
 '13,8 м²',
 '13,9 м²',
 '14 м²',
 '14,2 м²',
 '14,3 м²',
 '14,4 м²',
 '14,5 м²',
 '14,6 м²',
 '14,7 м²',
 '14,8 м²',
 '14,9 м²',
 '15 м²',
 '15,1 м²',
 '15,2 м²',
 '15,3 м²',
 '15,4 м²',
 '15,5 м²',
 '15,6 м²',
 '15,7 м²',
 '15,8 м²',
 '15,9 м²',
 '16 м²',
 '16,1 м²',
 '16,2 м²',
 '16,3 м²',
 '16,4 м²',
 '16,5 м²',
 '16,6 м²',
 '16,7 м²',
 '16,8 м²',
 '16,9 м²',
 '17 м²',
 '17,1 м²',
 '17,2 м²',
 '17,3 м²',
 '17,4 м²',
 '17,6 м²',
 '17,7 м²',
 '17,8 м²',
 '17,9 м²',
 '18 м²',
 '18,1 м²',
 '18,2 м²',
 '18,3 м²',
 '18,4 м²',
 '18,5 м²',
 '18,6 

Стоит сделать замечание, что появилась новая переменная, которая информирует об отделке конкретной квартиры, а не всех квартир в доме, поэтому очевидно включить в признаки только ее, также есть год сдачи каждой квартиры, что намного информативней, чем год сдачи дома, где помимо интрвалов, есть еще и категориальный признак "Сдан".

__Следующим этапом__ разберемся с площадями и приведем их в численный вид, затем разделим переменную этажи на две : этаж, на котором находится непосредственно квартира и суммарное количество этажей в доме

In [258]:
def fixing_square(list_to_fix):
    new_list = []
    for i in list_to_fix:
        if i != '':
            s = i.split()[0]
            if ',' in s:
                s = '.'.join(s.split(','))
            new_list.append(float(s))
        else:
            new_list.append('')
    return new_list

In [261]:
square_all_last = fixing_square(square_all)
square_liv_last = fixing_square(square_liv)
square_kitch_last = fixing_square(square_kitch)



In [263]:
square_all_last[:10]

[32.2, 48.06, 37.62, 42.8, 47.9, 44.9, '', 85.13, 48.73, 77.85]

In [264]:
square_liv_last[:10]

['', '', 12.8, '', '', 13.5, '', 33.9, 12.5, 30.5]

In [265]:
square_kitch_last[:10]

['', '', 3.1, '', '', 18.4, '', 6.2, 22.8, 27.1]

In [269]:
flat_floor = []
total_num_floor = []

for i in floors_flat_all:
    if i != '':
        i = i.split(' из ')
        flat_floor.append(int(i[0]))
        total_num_floor.append(int(i[1]))
    else:
        flat_floor.append('')
        total_num_floor.append('')

In [270]:
flat_floor[:10]

[5, 2, 2, 2, 2, 6, '', 8, 5, 5]

In [271]:
total_num_floor[:10]

[22, 26, 36, 45, 16, 19, '', 43, 8, 21]

###  Метро

Нам дан список из ближайших метро и время до них. Давайте выведем количество метро рядом, ближайшее метро и наименьшее время до метро.

In [303]:
metro[:10]

[['Савёловская\n13 мин.', 'Марьина роща\n17 мин.', 'Савёловская\n18 мин.'],
 ['Спартак\n7 мин.', 'Тушинская\n13 мин.', 'Трикотажная\n4 мин.'],
 ['Дмитровская\n8 мин.', 'Гражданская\n16 мин.', 'Тимирязевская\n3 мин.'],
 ['Фили\n5 мин.', 'Парк Победы\n12 мин.', 'Кутузовская\n17 мин.'],
 ['Белорусская\n4 мин.', 'Маяковская\n16 мин.', 'Менделеевская\n20 мин.'],
 ['Терехово\n1 мин.', 'Мнёвники\n4 мин.', 'Карамышевская\n5 мин.'],
 [],
 ['Дмитровская\n7 мин.', 'Гражданская\n17 мин.', 'Тимирязевская\n3 мин.'],
 ['Ленинский проспект\n13 мин.',
  'Площадь Гагарина\n15 мин.',
  'Воробьёвы горы\n19 мин.'],
 ['Белорусская\n7 мин.', 'Менделеевская\n18 мин.', 'Савёловская\n18 мин.']]

In [304]:
new_metro = list_to_dict(metro)
new_metro[0]

{'Савёловская': '18 мин.', 'Марьина роща': '17 мин.'}

In [296]:
number_stations = [len(x) for x in new_metro]
number_stations[:10]

[2, 3, 3, 3, 3, 3, 0, 3, 3, 3]

In [314]:
for i in new_metro:
    for key, value in i.items():
        if value.isdigit() == True:
            i[key] = int(value.split()[0])
        else:
            i[key] = value.split()[0]

In [316]:
new_metro[:10]

[{'Савёловская': 18, 'Марьина роща': 17},
 {'Спартак': 7, 'Тушинская': 13, 'Трикотажная': 4},
 {'Дмитровская': 8, 'Гражданская': 16, 'Тимирязевская': 3},
 {'Фили': 5, 'Парк Победы': 12, 'Кутузовская': 17},
 {'Белорусская': 4, 'Маяковская': 16, 'Менделеевская': 20},
 {'Терехово': 1, 'Мнёвники': 4, 'Карамышевская': 5},
 {},
 {'Дмитровская': 7, 'Гражданская': 17, 'Тимирязевская': 3},
 {'Ленинский проспект': 13, 'Площадь Гагарина': 15, 'Воробьёвы горы': 19},
 {'Белорусская': 7, 'Менделеевская': 18, 'Савёловская': 18}]

In [325]:
for i in new_metro:
    for key, value in i.items():
        if str(value).isdigit() == False:
            print(key, value)
# есть элементы словаря, где времени не указано, тк метро не открылось еще

Яхромская откроется
Яхромская откроется
Яхромская откроется
Яхромская откроется
Яхромская откроется
Яхромская откроется
Яхромская откроется
Яхромская откроется
Яхромская откроется


In [356]:
min_time = []
closest_station = []

for i in new_metro:
    if i != {}:
        if 'Яхромская' not in i.keys():
            minimum_time = min(i.values())
            for key, value in i.items():
                if value == min(i.values()):
                    closest_station.append(key)
                    min_time.append(value)
                    break
        else:
            i.pop('Яхромская')
            min_value = min(i.values())
            for key, value in i.items():
                if value == min_value:
                    min_time.append(value)
                    closest_station.append(value)
    else:
        min_time.append('')
        closest_station.append('')
            
            

In [357]:
min_time[:10]

[17, 4, 3, 5, 4, 1, '', 3, 13, 7]

In [354]:
closest_station[:10]

['Марьина роща',
 'Трикотажная',
 'Тимирязевская',
 'Фили',
 'Белорусская',
 'Терехово',
 '',
 'Тимирязевская',
 'Ленинский проспект',
 'Белорусская']

### Округ

In [335]:
okrug[:10]

[['Москва, СВАО, р-н Марьина роща, Полковая ул., 1С5На карте'],
 ['Москва, СЗАО, р-н Покровское-Стрешнево, Алиа жилой комплекс, к6На карте'],
 ['Москва, САО, р-н Савеловский, Симфония 34 жилой комплексНа карте'],
 ['Москва, ЗАО, р-н Дорогомилово, Кутузовский проезд, 16А/1На карте'],
 ['Москва, САО, р-н Беговой, Северный административный округ, Слава жилой комплекс, к2На карте'],
 ['Москва, СЗАО, р-н Хорошево-Мневники, проезд Проектируемый № 1077На карте'],
 [],
 ['Москва, САО, р-н Савеловский, 2-я Хуторская ул., 34На карте'],
 ['Москва, ЮЗАО, р-н Гагаринский, Лунар жилой комплексНа карте'],
 ['Москва, САО, р-н Беговой, ул. 1-я Ямского Поля, 28С3На карте']]

In [343]:
okrug_res = []
for i in okrug:
    if i != []:
        i = i[0].split(',')
        okrug_res.append(i[1])
    else:
        okrug_res.append('')

In [345]:
okrug_res[:10]

[' СВАО',
 ' СЗАО',
 ' САО',
 ' ЗАО',
 ' САО',
 ' СЗАО',
 '',
 ' САО',
 ' ЮЗАО',
 ' САО']

## Ура !!! Теперь запишем все признаки в табличку

In [381]:

print(len(num_rooms),
len(type_flat),
len(class_liv),
len(house_type),
len(parking),
len(square_all_last),
len(square_liv_last ),
len(square_kitch_last),
len(floors_flat_all ),
len(god_sdachi ),
len(decor_flat),
len(flat_floor),
len(total_num_floor),
len(min_time),
len(closest_station),
len(okrug_res),
     len(number_stations),len(name_jk_latest))

1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356 1356


In [382]:
features = {'Округ':okrug_res,'Ближайшая станция метро':closest_station, 'Время до метро':min_time, 'Метро рядом':number_stations,
            'Застройщик':build_company, 'ЖК': name_jk_latest,'Класс':class_liv,'Тип квартиры':type_flat,
            'Этаж': flat_floor, 'Этажей в доме': total_num_floor, 'Отделка':decor_flat,'Количество комнат':num_rooms,
            'Площадь квартиры':square_all_last,'Жилая площадь': square_liv_last , 'Площадь кухни':square_kitch_last,
            'Тип дома':house_type, 'Парковка':parking, 'Год сдачи': god_sdachi}

In [383]:
import pandas as pd
data = pd.DataFrame(features)

In [385]:
data['Цена'] = target

In [386]:
data

,Округ,Ближайшая станция метро,Время до метро,Метро рядом,Застройщик,ЖК,Класс,Тип квартиры,Этаж,Этажей в доме,Отделка,Количество комнат,Площадь квартиры,Жилая площадь,Площадь кухни,Тип дома,Парковка,Год сдачи,Цена
0,СВАО,Марьина роща,17,2,Компания Пионер,«PRIDE (Прайд)»,Премиум,квартира,5,22,Без отделки,1,32.2,,,Монолитный,Подземная,2025,16 106 665
1,СЗАО,Трикотажная,4,3,СК Резиденс,«ÁLIA (АЛИЯ)»,Бизнес,квартира,2,26,Неизвестно,1,48.06,,,Монолитно-кирпичный,"Подземная, гостевая",2023,16 122 800
2,САО,Тимирязевская,3,3,MR Group,«SYMPHONY 34 (Симфони 34)»,Премиум,квартира,2,36,Чистовая,1,37.62,12.8,3.1,Монолитный,"Подземная, гостевая",2025,18 472 662
3,ЗАО,Фили,5,3,Аеон Девелопмент,«River Park Towers Кутузовский (Ривер Парк Тау...,Премиум,квартира,2,45,Без отделки,1,42.8,,,Монолитно-кирпичный,Подземная,2026,22 450 632
4,САО,Белорусская,4,3,MR Group,«Комплекс апартаментов Slava (Слава)»,Премиум,апартаменты,2,16,Предчистовая,1,47.9,,,Монолитно-кирпичный,"Подземная, гостевая",2023,25 160 002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,,,,0,,,,,,,,,,,,,,,
1352,НАО (Новомосковский),Ольховая,14,3,Новые Ватутинки,«Новые Ватутинки. Микрорайон Центральный»,Комфорт,квартира,8,8,Чистовая,2,51.1,29.4,,"Панельный, монолитный","Отдельная многоуровневая, гостевая",2024,11 308 226
1353,ЮВАО,Дубровка,3,3,Level Group,«Level Южнопортовая»,Комфорт,квартира,2,69,Без отделки,2,35.4,10.6,16.5,Монолитный,Подземная,2025,11 327 234
1354,ЮВАО,Угрешская,2,3,Level Group,«Level Южнопортовая»,Комфорт,квартира,2,18,Без отделки,1,38.1,17.3,11.4,Монолитный,Подземная,2025,11 443 758


In [387]:
data.to_csv(r'cian_data.csv', index = False)

In [388]:
data.to_excel(r'cian_data.xlsx', index = False )